In [1]:
import csv

def loadFile(filename):
    with open(filename, 'r') as csvfile:
        rows = csv.reader(csvfile)
        dataset = list(rows)
        dataset.pop(0)
        return dataset

In [2]:
import random

def splitDataset(dataset, splitRatio):
    trainingSet = []
    size = int(len(dataset)*splitRatio)
    length = len(dataset[0])-1
    
    for i in range(size):
        item = random.choice(dataset)
        itemIndex = dataset.index(item)
        trainingSet.append(dataset.pop(itemIndex))
    
    return [trainingSet, dataset]

In [3]:
def getindex(Tuple):
    if Tuple[4]=='LookupError':
        return 0
    if Tuple[4]=='B':
        return 1
    if Tuple[4]=='R':
        return 2
    

In [4]:
def AVCSet(trainingSet):
    attributes = len(trainingSet[0])-1
    AVCSet = []
    instancesLen = len(trainingSet)
    for i in range(attributes):
        temp = [[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]]
        for j in range(instancesLen):
            ind1 = int(trainingSet[j][i])-1
            ind2 = getindex(trainingSet[j])
            temp[ind1][ind2]+=1
        AVCSet.append(temp)
    return AVCSet


In [5]:
def prior(trainingSet):
    totalInstances = [0,0,0]
    total = len(trainingSet)
    
    for i in trainingSet:
        ind = getindex(i)
        totalInstances[ind]+=1
        
    Prior = []
    for i in totalInstances:
        Prior.append( i/total )
    return [totalInstances, Prior]


In [6]:
def likelihood(avcset):
    Likelihood = []
    for i in range(len(avcset)):
        temp1 = []
        for j in range(len(avcset[i])):
            temp2 = []
            for k in range(len(avcset[i][j])):
                numerator = avcset[i][j][k]
                denominetor = totalInstances[k]
                temp2.append( numerator/denominetor )
            temp1.append(temp2)
        Likelihood.append(temp1)
    return Likelihood


In [7]:
def lkh(ind1, ind2, ind3):
    return Likelihood[ind1][ind2][ind3]


In [8]:
def predictedClass(Tuple, Prior):
    length = len(Tuple)-1
    
    probability = [1,1,1]
    for i in range(0,3):
        for t in range(length):
            probability[i] *= lkh(t, int(Tuple[t])-1, i)
        probability[i] *= Prior[i]
    
    pClassValue = max(probability[0], probability[1], probability[2])

    return probability.index(pClassValue)
#print(predictedClass(testSet[0]))

In [9]:
def predict(testSet, Prior):
    instancesLen = len(testSet)
    PredictedClass=[]
    
    for Tuple in testSet:
        PredictedClass.append(predictedClass(Tuple, Prior))
    return PredictedClass
predicted = predict(testSet, Prior)

NameError: name 'testSet' is not defined

In [ ]:
def confusionMatrix(testSet, predicted):
    ConfusionMatrix = [[0,0,0],[0,0,0],[0,0,0]]
    for i in range(len(predicted)):
        ind = getindex(testSet[i])
        ConfusionMatrix[ ind ][ predicted[i] ]+=1
        
    return ConfusionMatrix


In [ ]:
def result(testSet, ConfusionMatrix):
    n = len(testSet)
    
    accuracy = 0
    for i in range(0,3):
        for j in range(0,3):
            if i==j:
                accuracy += ConfusionMatrix[i][j]
    
    accuracy/=n
    
    recall = []
    
    for i in range(0,3):
        r = ConfusionMatrix[i][i]
        temp = ConfusionMatrix[i][0]+ConfusionMatrix[i][1]+ConfusionMatrix[i][2]
        if temp==0:
            recall.append(0)
        else:
            r /= (temp)
            recall.append(r)        
        
    percision = []
    for i in range(0,3):
        p = ConfusionMatrix[i][i]
        temp = ConfusionMatrix[0][i]+ConfusionMatrix[1][i]+ConfusionMatrix[2][i]
        if temp==0:
            percision.append(0)
        else:
            p /= (temp)
            percision.append(p)
    print("Accuracy", accuracy*100)
    print("Recall[L,B,R]= ", recall)
    print("Percision[L,B,R]= ", percision)


In [10]:
def main():
    dataset = loadFile('balance_scale_dataset.csv')
    trainingSet, testSet = splitDataset(dataset.copy(),.75)
    
    print("Instances of training set: ", len(trainingSet))
    print("Instances of test set: ", len(testSet))
    
    avcset = AVCSet(trainingSet)
    totalInstances, Prior = prior(trainingSet)
    Likelihood = likelihood(avcset)
    predicted = predict(testSet, Prior)
    ConfusionMatrix = confusionMatrix(testSet, predicted)
    
    print("ConfusionMatrix")
    for i in ConfusionMatrix:
        print(i)

    result(testSet, ConfusionMatrix)

In [11]:
main()

Instances of training set:  468
Instances of test set:  157


TypeError: list indices must be integers or slices, not NoneType